In [3]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
import plotly.graph_objects as go

from dustmaps.edenhofer2023 import Edenhofer2023Query

plot_save_directory = "./savedplots/"

In [4]:
import dustmaps.edenhofer2023
dustmaps.edenhofer2023.fetch()

Checking existing file to see if MD5 sum matches ...
File exists. Not overwriting.


In [5]:
def check_inside(x, y, z, r, h, t=(0,0,0), RotMat=np.array([[1,0,0],[0,1,0],[0,0,1]])):
    """
    Check if a point is inside the cylinder
    """
    # substraction of the translation
    x -= t[0]
    y -= t[1]
    z -= t[2]
    # rotation with the inverse of the rotation matrix
    x, y, z = np.linalg.inv(RotMat)@np.array([x, y, z])

    return z**2 + y**2 <= r**2 and -h/2 <= x <= h/2

In [6]:
def filledcylinder(r, h, nt=90, nh=100, nr=150):
    """
    Parametrize cylinder of radius r, height h
    """
    theta = np.linspace(0, 2*np.pi, nt)
    v = np.linspace(-h/2, h/2, nh)
    rr = np.linspace(0, r, nr)
    rr, theta, v = np.meshgrid(rr, theta, v)

    y = rr*np.cos(theta)
    z = rr*np.sin(theta)
    x = v
    return x, y, z

def rotationMatrix(a=0, b=0, c=0):
    """
    Return the Rotation Matrix with yaw a about z, pitch b about y, and roll c about x
    """
    Ra = np.array([
        [np.cos(a), -np.sin(a), 0],
        [np.sin(a), np.cos(a), 0],
        [0, 0, 1]])

    Rb = np.array([
        [np.cos(-b), 0, np.sin(-b)],
        [0, 1, 0],
        [-np.sin(-b), 0, np.cos(-b)]
    ])

    Rc = np.array([
        [1, 0, 0],
        [0, np.cos(c), -np.sin(c)],
        [0, np.sin(c), np.cos(c)]
    ])

    return Ra@Rb@Rc

In [7]:
def query_Edenhofer(x, y, z):
    xx, yy, zz = np.meshgrid(x, y, z)

    sc = SkyCoord(
        xx.flatten() * u.pc,
        yy.flatten() * u.pc,
        zz.flatten() * u.pc,
        frame = "galactic",
        representation_type = "cartesian"
    )
    sc.representation_type = "spherical"
    dust = Edenhofer2023Query().query(sc)
    return dust

In [8]:
def query_Edenhofer_in_cyl(t=(0, 0, 0), r=100, h=100, a=0, b=0, nt=20, nh=15, nr=20, nx=20, ny=20, nz=20):
    x1, y1, z1 = filledcylinder(r, h)
    RotMat = rotationMatrix(a, b)

    S = np.stack((x1, y1, z1), axis = 2)
    rotatedStack = RotMat@S
    xx = rotatedStack[:,:,0,:]+t[0]
    yy = rotatedStack[:,:,1,:]+t[1]
    zz = rotatedStack[:,:,2,:]+t[2]

    xt = np.linspace(np.min(xx), np.max(xx), nx)
    yt = np.linspace(np.min(yy), np.max(yy), ny)
    zt = np.linspace(np.min(zz), np.max(zz), nz)

    xt_grid, yt_grid, zt_grid = np.meshgrid(xt, yt, zt)

    sc = SkyCoord(
        xt_grid.flatten() * u.pc,
        yt_grid.flatten() * u.pc,
        zt_grid.flatten() * u.pc,
        frame = "galactic",
        representation_type = "cartesian"
    )
    sc.representation_type = "spherical"
    dust = Edenhofer2023Query().query(sc)

    points_in_space = np.stack([xt_grid.flatten(), yt_grid.flatten(), zt_grid.flatten(), dust], axis = 1)
    points_in_cylinder = np.stack([xt_grid.flatten(), yt_grid.flatten(), zt_grid.flatten(), np.zeros_like(dust)], axis = 1)

    for i in np.arange(0, dust.shape[0]):
        if check_inside(points_in_space[i,0], points_in_space[i,1], points_in_space[i,2], r, h, t, RotMat):
            points_in_cylinder[i,3] = points_in_space[i,3]
    return points_in_cylinder

In [9]:
def Volume_from_dust(points_in_space, minval = 1e-7, maxval = 5e-4, color = "red", num = 100):
    vol = go.Volume(
        x = points_in_space[:,0],
        y = points_in_space[:,1],
        z = points_in_space[:,2],
        value = points_in_space[:,3],
        isomin = minval,
        isomax = maxval,
        colorscale = [[0, "white"], [1, color]],
        name = "Cloud " + str(num),
        flatshading = True,
        showscale = False,
        opacityscale = [[0,0], [1,1]],
        surface = dict(show=True, count=1),
        showlegend = True
    )
    return vol

# Cloud 1

In [10]:
### Wolke 1

#Define SkyCoord
#x1 = np.arange(630, 870, 10)
#y1 = np.arange(-170, 150, 10)
#z1 = np.arange(0, 250,10)
#
#dust_W1 = query_Edenhofer(x1, y1, z1)

points_in_C1 = query_Edenhofer_in_cyl(
    t=(740,40,100),
    r=80,
    h=300,
    a=-(1/4)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [11]:
volume_dust_C1 = Volume_from_dust(
    points_in_C1,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "red",
    num = 1
)

___
# Cloud 2

In [12]:
### Wolke 2

#Define SkyCoord
#x2 = np.arange(450, 660, 10)
#y2 = np.arange(-440, -100, 10)
#z2 = np.arange(0, 220,10)
#
#dust_W2 = query_Edenhofer(x2, y2, z2)

points_in_C2 = query_Edenhofer_in_cyl(
    t=(530, -270, 110),
    r=90,
    h=345,
    a=(1/3)*np.pi,
    b=-(1/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [13]:
volume_dust_C2 = Volume_from_dust(
    points_in_C2,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "orange",
    num = 2
)

___
# Cloud 3

In [67]:
### Wolke 3

#Define SkyCoord
#x3 = np.arange(-850, -550, 10)
#y3 = np.arange(-390, 200, 10)
#z3 = np.arange(-80, 150,10)
#
#dust_W3 = query_Edenhofer(x3, y3, z3)

points_in_C3 = query_Edenhofer_in_cyl(
    t=(-775, 150, 0),
    r=90,
    h=800,
    a=-(1/2)*np.pi,
    b=0,
    ny=50
)

Optimizing map for querying (this might take a couple of seconds)...


In [68]:
volume_dust_C3 = Volume_from_dust(
    points_in_C3,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "blue",
    num = 3
)

___
# Cloud 4

In [16]:
### Wolke 4

#Define SkyCoord
#x4 = np.arange(240, 540, 10)
#y4 = np.arange(250, 800, 10)
#z4 = np.arange(-410, -130,10)
#
#dust_W4 = query_Edenhofer(x4, y4, z4)

points_in_C4 = query_Edenhofer_in_cyl(
    t=(425, 510, -260),
    r=90,
    h=550,
    a=(7.8/18)*np.pi,
    b=(2.4/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [17]:
volume_dust_C4 = Volume_from_dust(
    points_in_C4,
    minval = 1e-6,
    maxval = 7e-4,
    color = "green",
    num = 4
)

___
# Cloud 5

In [18]:
### Wolke 5

#Define SkyCoord
#x5 = np.arange(-550, -100, 10)
#y5 = np.arange(-650, 0, 10)
#z5 = np.arange(150, 550,10)
#
#dust_W5 = query_Edenhofer(x5, y5, z5)

points_in_C5 = query_Edenhofer_in_cyl(
    t=(-340, -320, 370),
    r=120,
    h=550,
    a=-(3.75/18)*np.pi,
    b=(4/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [19]:
volume_dust_C5 = Volume_from_dust(
    points_in_C5,
    minval = 1e-6,
    maxval = 2e-4,
    color = "cyan",
    num = 5
)

___
# Cloud 6

In [20]:
### Wolke 6

#Define SkyCoord
#x6 = np.arange(-1100, -690, 10)
#y6 = np.arange(-710, -490, 10)
#z6 = np.arange(-110, 260, 10)
#
#dust_W6 = query_Edenhofer(x6, y6, z6)

points_in_C6 = query_Edenhofer_in_cyl(
    t=(-910, -595, 80),
    r=95,
    h=500,
    a=(1/18)*np.pi,
    b=(4/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [21]:
volume_dust_C6 = Volume_from_dust(
    points_in_C6,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "teal",
    num = 6
)

___
# Cloud 7

In [22]:
### Wolke 7

#Define SkyCoord
#x7 = np.arange(-910, -600, 10)
#y7 = np.arange(-880, -730, 10)
#z7 = np.arange(170, 370,10)
#
#dust_W7 = query_Edenhofer(x7, y7, z7)

points_in_C7 = query_Edenhofer_in_cyl(
    t=(-750, -825, 230),
    r=65,
    h=250,
    a=(0.75/18)*np.pi,
    b=(3/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [23]:
volume_dust_C7 = Volume_from_dust(
    points_in_C7,
    minval = 1e-6,
    maxval = 1e-3,
    color = "gray",
    num = 7
)

___
# Cloud 8

In [24]:
### Wolke 8a

#Define SkyCoord
#x8a = np.arange(-430, -290, 10)
#y8a = np.arange(-1120, -1030, 10)
#z8a = np.arange(-50, 50, 10)
#
#dust_W8a = query_Edenhofer(x8a, y8a, z8a)

points_in_C8a = query_Edenhofer_in_cyl(
    t=(-360, -1075, 0),
    r=35,
    h=130,
    a=(1/18)*np.pi,
    b=-(1/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [25]:
### Wolke 8b

#Define SkyCoord
#x8b = np.arange(-575, -305, 10)
#y8b = np.arange(-1140, -960, 10)
#z8b = np.arange(-120, -20, 10)
#
#dust_W8b = query_Edenhofer(x8b, y8b, z8b)

points_in_C8b = query_Edenhofer_in_cyl(
    t=(-440, -1050, -70),
    r=40,
    h=260,
    a=(2.5/18)*np.pi,
    b=-(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [26]:
### Wolke 8c

#Define SkyCoord
#x8c = np.arange(-725, -345, 10)
#y8c = np.arange(-1140, -820, 10)
#z8c = np.arange(-190, 10, 10)
#
#dust_W8c = query_Edenhofer(x8c, y8c, z8c)

points_in_C8c = query_Edenhofer_in_cyl(
    t=(-535, -980, -90),
    r=85,
    h=340,
    a=(3/18)*np.pi,
    b=(0.45/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [27]:
volume_dust_C8a = Volume_from_dust(
    points_in_C8a,
    minval = 1e-6,
    maxval = 1e-3,
    color = "yellow",
    num = 8
)
volume_dust_C8a.name = volume_dust_C8a.name + "a"

In [28]:
volume_dust_C8b = Volume_from_dust(
    points_in_C8b,
    minval = 1e-6,
    maxval = 2e-3,
    color = "yellow",
    num = 8
)
volume_dust_C8b.name = volume_dust_C8b.name + "b"

In [29]:
volume_dust_C8c = Volume_from_dust(
    points_in_C8c,
    minval = 1e-7,
    maxval = 1e-3,
    color = "yellow",
    num = 8
)
volume_dust_C8c.name = volume_dust_C8c.name + "c"

___
# Cloud 9

In [30]:
### Wolke 9

#Define SkyCoord
#x9 = np.arange(-400, -190, 10)
#y9 = np.arange(690, 950, 10)
#z9 = np.arange(-70, 200, 10)
#
#dust_W9 = query_Edenhofer(x9, y9, z9)

points_in_C9 = query_Edenhofer_in_cyl(
    t=(-140, 920, 120),
    r=120,
    h=430,
    a=(4.5/18)*np.pi,
    b=0
)

Optimizing map for querying (this might take a couple of seconds)...


In [31]:
volume_dust_C9 = Volume_from_dust(
    points_in_C9,
    minval = 1e-7,
    maxval = 2e-3,
    color = "lightgreen",
    num = 9
)

___
# Cloud 10

In [32]:
### Wolke 10

#Define SkyCoord
#x10 = np.arange(-980, -800, 10)
#y10 = np.arange(280, 440, 10)
#z10 = np.arange(-480, -270, 10)
#
#dust_W10 = query_Edenhofer(x10, y10, z10)

points_in_C10 = query_Edenhofer_in_cyl(
    t=(-910, 350, -380),
    r=60,
    h=170,
    a=(3/18)*np.pi,
    b=-(6.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [33]:
volume_dust_C10 = Volume_from_dust(
    points_in_C10,
    minval = 1e-7,
    maxval = 5e-4,
    color = "chocolate",
    num = 10
)

volume_dust_C10.name += " (Peanut)"

___
# Cloud 11

In [34]:
### Wolke 11

#Define SkyCoord
#x11 = np.arange(-350, -150, 10)
#y11 = np.arange(90, 250, 10)
#z11 = np.arange(200, 310, 10)
#
#dust_W11 = query_Edenhofer(x11, y11, z11)

points_in_C11 = query_Edenhofer_in_cyl(
    t=(-260, 150, 240),
    r=60,
    h=185,
    a=(5/18)*np.pi,
    b=(1/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [35]:
volume_dust_C11 = Volume_from_dust(
    points_in_C11,
    minval = 1e-7,
    maxval = 9e-4,
    color = "gold",
    num = 11
)

___
# Cloud 12

In [36]:
### Wolke 12

#Define SkyCoord
#x12 = np.arange(-550, -350, 10)
#y12 = np.arange(70, 250, 10)
#z12 = np.arange(-120, 0, 10)

#dust_W12 = query_Edenhofer(x12, y12, z12)

points_in_C12 = query_Edenhofer_in_cyl(
    t=(-430, 150, -70),
    r=60,
    h=220,
    a=(2.2/18)*np.pi,
    b=-(0.25/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [37]:
volume_dust_C12 = Volume_from_dust(
    points_in_C12,
    minval = 9e-6,
    maxval = 5e-3,
    color = "olive",
    num = 12
)

___
# Cloud 13

In [38]:
### Wolke 13

#Define SkyCoord
#x13 = np.arange(150, 550, 10)
#y13 = np.arange(-30, 260, 10)
#z13 = np.arange(-450, -190, 10)
#
#dust_W13 = query_Edenhofer(x13, y13, z13)

points_in_C13 = query_Edenhofer_in_cyl(
    t=(320, 130, -290),
    r=100,
    h=300,
    a=-(2.7/18)*np.pi,
    b=-(1.5/18)*np.pi
)   

Optimizing map for querying (this might take a couple of seconds)...


In [39]:
volume_dust_C13 = Volume_from_dust(
    points_in_C13,
    minval = 1e-6,
    maxval = 4e-4,
    color = "aquamarine",
    num = 13
)

___
# Cloud 14

In [40]:
### Wolke 14

#Define SkyCoord
#x14 = np.arange(-440, -160, 10)
#y14 = np.arange(755, 1285, 10)
#z14 = np.arange(140, 380, 10)
#
#dust_W14 = query_Edenhofer(x14, y14, z14)

points_in_C14 = query_Edenhofer_in_cyl(
    t=(-300, 1020, 265),
    r=100,
    h=500,
    a=(8/18)*np.pi,
    b=-(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [41]:
volume_dust_C14 = Volume_from_dust(
    points_in_C14,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "aquamarine",
    num = 14
)

___
# Cloud 15

In [42]:
### Wolke 15

#Define SkyCoord
#x15 = np.arange(-310, -30, 10)
#y15 = np.arange(-690, -420, 10)
#z15 = np.arange(-80, 80, 10)
#
#dust_W15 = query_Edenhofer(x15, y15, z15)

points_in_C15 = query_Edenhofer_in_cyl(
    t=(-170, -570, -20),
    r=110,
    h=200,
    a=(9/18)*np.pi,
    b=0
)

Optimizing map for querying (this might take a couple of seconds)...


In [43]:
volume_dust_C15 = Volume_from_dust(
    points_in_C15,
    minval = 1e-6,
    maxval = 4e-4,
    color = "deepskyblue",
    num = 15
)

___
# Cloud 16S

In [44]:
### Wolke 16

#Define SkyCoord
#x16 = np.arange(-1010, -850, 10)
#y16 = np.arange(50, 210, 10)
#z16 = np.arange(-290, -130, 10)
#
#dust_W16 = query_Edenhofer(x16, y16, z16)

points_in_C16 = query_Edenhofer_in_cyl(
    t=(-920, 120, -215),
    r=100,
    h=120,
    a=0,
    b=0
)

Optimizing map for querying (this might take a couple of seconds)...


In [45]:
volume_dust_C16 = Volume_from_dust(
    points_in_C16,
    minval = 1e-6,
    maxval = 8e-4,
    color = "violet",
    num = 16
)

___
# Cloud 17

In [46]:
### Wolke 17

#Define SkyCoord
#x17 = np.arange(-520, 160, 10)
#y17 = np.arange(-230, 330, 10)
#z17 = np.arange(-410, -70, 10)
#
#dust_W17 = query_Edenhofer(x17, y17, z17)

points_in_C17 = query_Edenhofer_in_cyl(
    t=(-180, 50, -250),
    r=100,
    h=700,
    a=(3.5/18)*np.pi,
    b=(1/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [47]:
volume_dust_C17 = Volume_from_dust(
    points_in_C17,
    minval = 1e-6,
    maxval = 1e-3,
    color = "fuchsia",
    num = 17
)

___
# Cloud 18

In [48]:
### Wolke 18

#Define SkyCoord
#x18 = np.arange(70, 330, 10)
#y18 = np.arange(410, 740, 10)
#z18 = np.arange(80, 290, 10)
#
#dust_W18 = query_Edenhofer(x18, y18, z18)

points_in_C18 = query_Edenhofer_in_cyl(
    t=(195, 600, 230),
    r=105,
    h=325,
    a=(8/18)*np.pi,
    b=-(3.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [49]:
volume_dust_C18 = Volume_from_dust(
    points_in_C18,
    minval = 1e-6,
    maxval = 6e-4,
    color = "crimson",
    num = 18
)

___
# Cloud 19(Vela)

In [50]:
### Wolke 19

#Define SkyCoord
#x19 = np.arange(-380, 290, 10)
#y19 = np.arange(-1210, -540, 10)
#z19 = np.arange(-200, 200, 10)
#
#dust_W19 = query_Edenhofer(x19, y19, z19)

points_in_C19 = query_Edenhofer_in_cyl(
    t=(-45, -875, 0),
    r=200,
    h=550,
    a=(4.5/18)*np.pi,
    b=0
)

Optimizing map for querying (this might take a couple of seconds)...


In [51]:
volume_dust_C19 = Volume_from_dust(
    points_in_C19,
    minval = 1e-7,
    maxval = 2e-3,
    color = "sienna",
    num = 19
)

___
# Cloud 20

In [52]:
### Wolke 20

points_in_C20 = query_Edenhofer_in_cyl(
    t=(960, 200, 190),
    r=100,
    h=420,
    a=-(4.5/18)*np.pi,
    b=-(1/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [53]:
volume_dust_C20 = Volume_from_dust(
    points_in_C20,
    minval = 1e-6,
    maxval = 6e-4,
    color = "sienna",
    num = 20
)

___
# Cloud 21

In [54]:
### Wolke 21a

points_in_C21a = query_Edenhofer_in_cyl(
    t=(-980, -190, 0),
    r=70,
    h=340,
    a=(8.4/18)*np.pi,
    b=(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [64]:
### Wolke 21b

points_in_C21b = query_Edenhofer_in_cyl(
    t=(-1040, -40, 80),
    r=55,
    h=420,
    a=(8.4/18)*np.pi,
    b=-(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [56]:
### Wolke 21c

points_in_C21c = query_Edenhofer_in_cyl(
    t=(-1070, 40, -60),
    r=85,
    h=460,
    a=(8.7/18)*np.pi,
    b=-(1.15/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [57]:
volume_dust_C21a = Volume_from_dust(
    points_in_C21a,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "peru",
    num = 21
)
volume_dust_C21a.name = volume_dust_C21a.name + "a"

In [65]:
volume_dust_C21b = Volume_from_dust(
    points_in_C21b,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "peru",
    num = 21
)
volume_dust_C21b.name = volume_dust_C21b.name + "b"

In [59]:
volume_dust_C21c = Volume_from_dust(
    points_in_C21c,
    minval = 1e-6,
    maxval = 1.6e-3,
    color = "peru",
    num = 21
)
volume_dust_C21c.name = volume_dust_C21c.name + "c"

___
# Cloud 22

In [70]:
### Wolke 22

points_in_C22 = query_Edenhofer_in_cyl(
    t=(20, 170, -60),
    r=55,
    h=200,
    a=(0.8/18)*np.pi,
    b=-(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [71]:
volume_dust_C22 = Volume_from_dust(
    points_in_C22,
    minval = 1e-6,
    maxval = 1e-3,
    color = "seagreen",
    num = 22
)

___
# Cloud 23

In [73]:
### Wolke 23

points_in_C23 = query_Edenhofer_in_cyl(
    t=(80, 190, 40),
    r=40,
    h=200,
    a=-(1.7/18)*np.pi,
    b=(0.5/18)*np.pi
)

Optimizing map for querying (this might take a couple of seconds)...


In [76]:
volume_dust_C23 = Volume_from_dust(
    points_in_C23,
    minval = 1e-6,
    maxval = 8e-4,
    color = "darkslategray",
    num = 23
)

___
# Full Galaxy Dust?

In [60]:
#Define SkyCoord
xf = np.arange(-1250, 1250, 50)
yf = np.arange(-1250, 1250, 50)
zf = np.arange(-1250, 1250, 50)

dust_f = query_Edenhofer(xf, yf, zf)

Optimizing map for querying (this might take a couple of seconds)...


In [61]:
xx, yy, zz = np.meshgrid(xf, yf, zf)
volume_dust_f = go.Volume(
    x = xx.flatten(),
    y = yy.flatten(),
    z = zz.flatten(),
    value = dust_f,
    flatshading = True,
    #opacity = 0.1,
    opacityscale = [[0, 0], [1, 0.5]],
    isomin = 2e-6,
    isomax = 6e-4,
    surface = dict(show=True, count=1),
    colorscale = [[0, "white"], [1, "black"]],
    visible = "legendonly",
    showlegend = True,
    showscale = False,
    name = "Galactic Plane",
    #contour = dict(show=True, width=4)
    #spaceframe = dict(show=True)
)

___
# Plotting

In [77]:
#Bounds manuell setten, vllt automatisier ich das noch, jz nur test
x_bounds = [-1250, 1250]
y_bounds = [-1250, 1250]
z_bounds = [-500, 600]

yscale = (np.max(y_bounds)-np.min(y_bounds))/(np.max(z_bounds)-np.min(z_bounds))
xscale = (np.max(x_bounds)-np.min(x_bounds))/(np.max(z_bounds)-np.min(z_bounds))

layout_3d = go.Layout(
    template = "plotly_white",
    #paper_bgcolor = "black",
    #plot_bgcolor = "black",
    title = "Dust",
    showlegend = True,
    scene = dict(
        aspectmode = 'manual',
        aspectratio = dict(x = xscale, y = yscale, z = 1),
        xaxis = dict(title = "x", range = x_bounds),
        yaxis = dict(title = "y", range = y_bounds),
        zaxis = dict(title = "z", range = z_bounds),
    )
)

sun_scatter3D = go.Scatter3d(
    x = [0],
    y = [0],
    z = [0],
    mode = 'markers',
    marker = dict(
        size = 3,
        color = 'yellow'
    ),
    name = 'Sun',
    hovertext = 'Sun'
)


fig_3d_dust = go.Figure(data = [
    sun_scatter3D,
    volume_dust_C1,
    volume_dust_C2,
    volume_dust_C3,
    volume_dust_C4,
    volume_dust_C5,
    volume_dust_C6,
    volume_dust_C7,
    volume_dust_C8a, volume_dust_C8b, volume_dust_C8c,
    volume_dust_C9,
    volume_dust_C10,
    volume_dust_C11,
    volume_dust_C12,
    volume_dust_C13,
    volume_dust_C15,
    volume_dust_C16,
    volume_dust_C17,
    volume_dust_C18,
    volume_dust_C19,
    volume_dust_C20,
    volume_dust_C21a, volume_dust_C21b, volume_dust_C21c,
    volume_dust_C22,
    volume_dust_C23,
    volume_dust_f
], layout = layout_3d)
fig_3d_dust.write_html(plot_save_directory + "test2_2.html")